In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
import src.cleanfun as cf

In [2]:
#cargamos csv
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
# sharks.shape

In [4]:
sharks.sample(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
15221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1826,2002.09.21.b,21-Sep-2002,2002.0,Unprovoked,USA,Oregon,Cape Kiwanda,Boogie boarding or Surfing,Garry Turner,M,...,2.4 m [8'] shark,"R. Collier, GSAF",2002.09.21.b-GarryTurner.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2002.09.21.b,2002.09.21.b,4477.0,NaN,NaN
13190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sharks.Type.value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [6]:
#eliminamos todas las columnas y filas que todos sus valores sean nulos
sharks.dropna(how="all", inplace=True)
sharks.dropna(how="all",axis = 1, inplace=True)

# Hipótesis
- hipotesis 1 los ataques de tiburones han icrmentado según han ido avanzando los años
- hipótesis 2 los ataquesde tiburones atacan suelen atacar a embarcaciones
- [hipóstesis 3 Florida es la capital mundial de los ataques de tiburones](https://www.lavanguardia.com/ocio/viajes/20210407/6631447/6-playas-mas-peligrosas-mundo.html)
- [hipótesis 4 ¿Es posible el ataque de tiburón en la costa española?](https://www.mundo-geo.es/naturaleza/es-posible-ataque-tiburon-en-costa-espanola_238643_102.html)

In [7]:
#compruebo qué colmunas tienen datos interesantes para mis hipótesis y borro las que no me hacen ninguna falta.
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [8]:
columnas_no = ['Case Number','Investigator or Source','pdf', 'href formula',  'href',  'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23'] 
sharks.drop(columnas_no, axis=1, inplace=True)

In [9]:
#ahora que hemos borrado columnas volvemos a hacer una pasada a los registros cuyos datos son todos nulos y borramos estas filas
sharks.dropna(how="all", inplace=True)

In [10]:
#cambio nombre de algunas columnas que tienen espacios o caracteres especiales en el nombre
#"Sex " "Species " "Fatal (Y/N)"
cols = ["Sex ","Species ","Fatal (Y/N)"]
new_names = {"Sex " : "Sex",
                 "Species " : "Species",
                 "Fatal (Y/N)" : "Fatal"}
sharks.rename(columns = new_names, inplace=True)

In [11]:
#segundo repaso a datos para ver si borramos alguna columna más que no me de información necesaria para las hipótesis
sharks.sample(20)
sharks.drop(["Name"],axis=1, inplace=True)

### Limpieza de datos nulos

In [12]:
#miramos cuantos datos nulos hay en cda columna.
sharks.isna().sum()

Date           0
Year           2
Type           4
Country       50
Area         455
Location     540
Activity     544
Sex          565
Age         2831
Injury        28
Fatal        539
Time        3354
Species     2838
dtype: int64

#### rellenamos los nulos en función de los datos que contienen las columnas


In [13]:
#Year
sharks.Year.unique()
#rellenamos años vacíos con 0
sharks.Year.fillna(0.0,inplace= True)
#convertirmos tipo de dato (float) a int.
sharks.Year = sharks.Year.astype(dtype = "int64")

In [14]:
#type
sharks.Type.unique() #['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable','Sea Disaster', nan, 'Boat', 'Boatomg']
sharks[sharks["Type"] == "Invalid"].sample(20)
#invalid NO nos sirve para los nulos porque es para ataques que no fueron de tiburón
sharks.Type.fillna("UNKNOWN",inplace= True)
sharks.Type = sharks.Type.str.strip() #borramos espacios
sharks = sharks[sharks["Type"] != "Invalid"] #eliminamos los registros que que equivalen a "Invalid" ya que no son ataques de tibruón
#nos da mucha información si el ataque fue en embarcación o direcgtamente a "nadador"

In [15]:
#Country, Area y Location -> nans == UNKNOWN
sharks["Country"].fillna("UNKNOWN", inplace = True)
sharks["Area"].fillna("UNKNOWN", inplace = True)
sharks["Location"].fillna("UNKNOWN", inplace = True)
sharks.Country = sharks.Country.str.strip()
sharks.Area = sharks.Area.str.strip()
sharks.Location = sharks.Location.str.strip()

In [16]:
#activity
list(sharks.Activity.unique())
#de esta columna podemos sacar datos sobre si el tiburón atacó a embarcación o persona.
sharks["Activity"].fillna("UNKNOWN", inplace = True)
sharks.Activity = sharks.Activity.str.strip()

In [17]:
#sex
sharks.Sex.unique()
#si el sexo de la víctima es desconocido lo rellenamos con "X"
#además queremos que se queden tres datos únicos -> M,F,X
sharks.Sex.fillna("X",inplace = True)
sharks.Sex = sharks.Sex.str.strip() #borramos espacios vacíos antes y después de cada cadena.
sharks.Sex.unique() #['F', 'M', 'X', 'lli', 'N', '.'] > lli, N y . == X
sharks["Sex"].replace('lli',"X",inplace=True)
sharks["Sex"].replace('.',"X",inplace=True)
sharks["Sex"].replace('N',"X",inplace=True)

In [18]:
#Age
sharks.Age.unique()
#rellenamos nans con "UNKNOWN" porque 0 puede confundir la media si luego necesitamos usarla.
sharks.Age.fillna("UNKNOWN",inplace= True)
sharks.Age = sharks.Age.str.strip()

In [19]:
#Injury > nan = UNKNOWN
sharks.Injury.unique()
sharks["Injury"].fillna("UNKNOWN", inplace = True)

In [20]:
#Fatal
sharks.Fatal.unique
#nulos = UNKNOWN
sharks["Fatal"].fillna("UNKNOWN", inplace = True)
sharks.Fatal = sharks.Fatal.str.strip()
sharks.Fatal.unique() #comprobar registros con FAtal == 'M', '2017', 'y'
sharks[(sharks["Fatal"] == 'M')|(sharks["Fatal"] == "2017")|(sharks["Fatal"] == "y")].Injury.unique
sharks["Fatal"].replace('M',"N",inplace=True)
sharks["Fatal"].replace('y',"Y",inplace=True)
sharks["Fatal"].replace('2017',"N",inplace=True)
sharks.Fatal.unique() #['N', 'Y', 'UNKNOWN']

array(['N', 'Y', 'UNKNOWN'], dtype=object)

In [21]:
#Time
sharks.Time.unique()
#nulos = UNKNOWN
sharks["Time"].fillna("UNKNOWN", inplace = True)
#puede que esta columna no la utlicemos para ningún análisis. No nos interesan los datos tan exactos de la hora del ataque.

In [22]:
#Species
list(sharks.Species.unique())
#nulos = UNKNOWN
sharks["Species"].fillna("UNKNOWN", inplace = True)
sharks.Species = sharks.Species.str.strip()

In [23]:
sharks.isna().sum()

Date        0
Year        0
Type        0
Country     0
Area        0
Location    0
Activity    0
Sex         0
Age         0
Injury      0
Fatal       0
Time        0
Species     0
dtype: int64

### Procesamos el texto de las columnas

##### Date, Year.

In [24]:
sharks.Date.value_counts()
#de la columna Date extraemos años y meses para nuevas columnas
sharks["Year_D"] = sharks["Date"].str.extract(r'(\d{4})')
sharks["Month_D"] = sharks["Date"].str.extract(r'([A-Z][a-z][a-z]-\d{4})')
sharks["Month_D"] = sharks["Month_D"].str.extract(r'([A-Z][a-z][a-z])')
#volvemos a rellenar los nulos que quedan 
#sharks["Date"].fillna("UNKNOWN", inplace = True)
sharks.Year_D.fillna("UNKNOWN",inplace= True)
sharks.Month_D.fillna("UNKNOWN",inplace= True)
#convertimos los strings de Year_D enteros
sharks.Year_D = sharks.Year_D.apply(cf.entero_if)
sharks["Month_D"] = sharks["Month_D"].str.upper()

##### Type

In [25]:
sharks.Type.unique()

array(['Boating', 'Unprovoked', 'Provoked', 'Questionable',
       'Sea Disaster', 'UNKNOWN', 'Boat', 'Boatomg'], dtype=object)

In [26]:
#Boating, Boat y Boatomg == mismo dato "Boat"
sharks["Type"] = sharks["Type"].str.replace((".*[Bb](OAT|oat).*"),"Boat", regex=True)
sharks.Type.unique()

array(['Boat', 'Unprovoked', 'Provoked', 'Questionable', 'Sea Disaster',
       'UNKNOWN'], dtype=object)

#####  Activity

In [27]:
#creamos una nueva columna "Act" strings que acaban en "ing" de "Activity"
sharks["Act"] = sharks.Activity.str.extract(r'(\w+ing)+')
#rellenamos los nuevos nans
sharks["Act"].fillna("UNKONWN", inplace = True)
sharks["Act"]= sharks["Act"].str.upper()

In [28]:
sharks.Act.value_counts().head(10)

SURFING         1041
SWIMMING         994
UNKONWN          761
FISHING          672
DIVING           470
SPEARFISHING     376
BATHING          175
WADING           157
BOARDING         130
STANDING         129
Name: Act, dtype: int64

###### Activity -> "Fishing"
- como en Type tenemos "Boat" para saber si hay ataques direcatemente a los barcos, pero luego en activity "Fishing" la mayoría son desde barcos, vamos a crear una columna "Boat" incluyendo todos los registros que son type "Boat" además de los que no son "Boat" pero son Fishin

In [29]:
#filtramos todos registros que son tipo Boat y de los que no son tipo Boat pero son activity fishing para ver cuantos realemnte van en barco
sharks["Boat"] = np.where(((sharks["Type"] == "Boat")|((sharks["Type"] != "Boat") & (sharks["Act"] == "FISHING"))),"Yes","No")

In [30]:
sharks.Boat.value_counts()

No     4918
Yes     837
Name: Boat, dtype: int64

#####  Sex

In [31]:
sharks.Sex.unique() #no hacemos nada más

array(['F', 'M', 'X'], dtype=object)

#####  Age

In [32]:
#dejamos solos los valores de edades de Age:
sharks.Age = sharks.Age.apply(cf.edades)
sharks.Age.unique()

array(['57', '11', 'UNKNOWN', '18', '52', '15', '12', '32', '10', '34',
       '30', '60', '33', '29', '54', '41', '37', '19', '25', '69', '38',
       '55', '35', '45', '40', '28', '20', '24', '26', '49', '14', '22',
       '7', '31', '17', '13', '42', '3', '50', '46', '16', '82', '48',
       '21', '51', '39', '58', '47', '61', '65', '73', '36', '66', '43',
       '9', '72', '59', '6', '64', '23', '71', '44', '27', '62', '68',
       '63', '70', '18 months', '53', '8', '77', '74', '56', '5', '86',
       '84', '87', '75', '9 months', '1', '81', '78', '2'], dtype=object)

#####  Injury

In [33]:
#creamos nueva columna con un resumen de tipos de lesiones
sharks["Injur_Type"] = sharks.Injury.apply(cf.actividades)
sharks["Injur_Type"] = sharks["Injur_Type"].str.upper()

In [34]:
sharks.Injur_Type.unique()

array(['NO INJURY', 'MINOR INJURIES', 'LACERATIONS', 'FATAL INJURY',
       'BITTEN', 'INJURED', 'OTHERS', 'PUNCTURED', 'LIMB SEVERED',
       'NO DETAILS', 'UNKNOWN'], dtype=object)

##### Fatal

In [35]:
sharks.Fatal.unique() #no hacemos nada más

array(['N', 'Y', 'UNKNOWN'], dtype=object)

##### Time
no vamos a usar esta columna, porque para nuestros datos no nos intersa a qué hora pasaron los incidentes

#####  species

In [36]:
#extraemos en una nueva columna en la que extraemos las especies de los tiburones desde la columna "Species"
sharks["Spec"] = sharks.Species.str.extract(r'(\w{3,}(?= shark))')

In [37]:
sharks["Spec"].fillna("UNKONWN", inplace = True)
sharks["Spec"] = sharks["Spec"].str.upper()

In [38]:
sharks.Spec.value_counts().head(5)

UNKONWN    3778
WHITE       627
TIGER       258
BULL        176
NURSE        97
Name: Spec, dtype: int64

#####  country // Areas

In [39]:
sharks["Country"] = sharks["Country"].str.upper()
sharks["Area"] = sharks["Area"].str.upper()
sharks.Area.value_counts().head(10) #las areas nos interesan sobre todo Florida en comparación con el resto...

FLORIDA                  971
NEW SOUTH WALES          436
UNKNOWN                  408
QUEENSLAND               287
HAWAII                   263
CALIFORNIA               260
WESTERN CAPE PROVINCE    185
KWAZULU-NATAL            171
WESTERN AUSTRALIA        170
EASTERN CAPE PROVINCE    148
Name: Area, dtype: int64

## Exportación

In [40]:
sharks.sample(5)

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species,Year_D,Month_D,Act,Boat,Injur_Type,Spec
945,17-Feb-2011,2011,Unprovoked,AUSTRALIA,SOUTH AUSTRALIA,Off Perforated Island near Coffin Bay,Diving for abalone,M,49,FATAL,Y,18h20,White shark x 2,2011,FEB,DIVING,No,FATAL INJURY,WHITE
4328,19955,1954,Unprovoked,ITALY,LIGUARIA,Finale Ligure,Spearfishing,M,27,Abdomen injured,N,UNKNOWN,UNKNOWN,1995,UNKNOWN,SPEARFISHING,No,INJURED,UNKONWN
3494,18-Dec-1967,1967,Unprovoked,FIJI,UNKNOWN,UNKNOWN,UNKNOWN,M,27,Arm lacerated,N,09h00,UNKNOWN,1967,DEC,UNKONWN,No,LACERATIONS,UNKONWN
1029,03-May-2010,2010,Unprovoked,MADAGASCAR,ANTSIRANANA PROVINCE,"Ambatolaoka, Nosy Be Island",Scuba diving,M,59,Lacerations to arm & chest,N,09h30,UNKNOWN,2010,MAY,DIVING,No,LACERATIONS,UNKONWN
3425,Aug-1969,1969,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,M,25,Am lacerated,N,UNKNOWN,UNKNOWN,1969,AUG,UNKONWN,No,LACERATIONS,UNKONWN


In [41]:
#exportarmos sharky
sharks.to_csv("data/sharks.csv")